In [23]:
import pandas as pd
from llama_index.llms.ollama import Ollama
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import FunctionTool
from typing import List, Dict
import json
import os

# Caricamento delle variabili d'ambiente
from dotenv import load_dotenv
from llama_index.llms.openai import OpenAI
load_dotenv()
os.getenv("OPENAI_API_KEY")
llm = OpenAI(model="gpt-3.5-turbo")

# ----------------------------
# FUNZIONE DI FILTRAGGIO DATI
# ----------------------------
def filtra_dati_per_chiavi(names: List[str], chiavi=List[str]) -> List[Dict]:
    """ Filtra il file JSONL e restituisci le informazioni in base alla chave 'denominazione' indicata.
        La funzione accetta una lista di chiavi e restituisce una lista di dizionari con i dati corrispondenti.
        Usa questo tool per ottenere informazioni specifiche sulle stazioni appaltanti.
    """

    from setting import Setting
    path_to_json = Setting().path_to_json

    with open(path_to_json, "r", encoding="utf-8") as f:
        data = [json.loads(line) for line in f][:10]

    list_risultati = []
    for names in names:
        for row in data:
            if row.get("denominazione") == names:
                risultato = {key: row.get(key) for key in chiavi}
                list_risultati.append(risultato)
    return list_risultati


# ----------------------------
# WRAPPER TOOL per LlamaIndex
# ----------------------------
filtra_dati_tool = FunctionTool.from_defaults(
    fn=filtra_dati_per_chiavi,
    name="filtra_dati_per_chiavi",
    description=(
        "Filtra il file JSONL e restituisci le informazioni in base alla chave 'denominazione' indicata. "
        "La funzione accetta una lista di chiavi e restituisce una lista di dizionari con i dati corrispondenti. "
        "Usa questo tool per ottenere informazioni specifiche sulle stazioni appaltanti."
    ),
)

# ----------------------------
# LLM setup
# ----------------------------
#from setting import Setting
#params = Setting()
#llm = Ollama(
#    model=params.model_name,
#    request_timeout=params.request_timeout,
#    json_mode=params.json_mode,
#    context_window=params.context_window,
#)

# ----------------------------
# AGENTE
# ----------------------------
agente = ReActAgent.from_tools(
    tools=[filtra_dati_tool],
    llm=llm,
    name="StazioniAppaltantiAgent",
    description="Un agente per estrarre le informazioni sulle stazioni appaltanti.",
    system_prompt=(
        "Sei un agente specializzato nell'estrazione di informazioni da un dataset di stazioni appaltanti. "
        "Rispondi sempre in Italiano." #Utilizza sempre i tools che hai a disposizione. 
    ),
    verbose=True,
)

# ----------------------------
# ESECUZIONE ESEMPIO
# ----------------------------
if __name__ == "__main__":
    risposta = agente.chat(
        "Dammi la PARTITA IVA del COMUNE DI SARONNO e della SCUOLA ALLIEVI CARABINIERI DI TORINO. Dammi anche l' indirizzo odonimo. Aggiungi descrizione delle due denomizaioni."
    )
    print(' ')
    print(risposta)

/opt/homebrew/Caskroom/miniconda/base/envs/agent/lib/python3.11/site-packages/llama_index/core/agent/react/base.py:154: DeprecationWarning: Call to deprecated class ReActAgent. (ReActAgent has been rewritten and replaced by llama_index.core.agent.workflow.ReActAgent.

This implementation will be removed in a v0.13.0 and the new implementation will be promoted to the `from llama_index.core.agent import ReActAgent` path.

See the docs for more information: https://docs.llamaindex.ai/en/stable/understanding/agent/)
  return cls(
/opt/homebrew/Caskroom/miniconda/base/envs/agent/lib/python3.11/site-packages/deprecated/classic.py:184: DeprecationWarning: Call to deprecated class AgentRunner. (AgentRunner has been deprecated and is not maintained.

This implementation will be removed in a v0.13.0.

See the docs for more information on updated agent usage: https://docs.llamaindex.ai/en/stable/understanding/agent/)
  return old_new1(cls, *args, **kwargs)
/opt/homebrew/Caskroom/miniconda/base/en

> Running step de1f7d70-76ee-4dd3-acfe-9801582c4813. Step input: Dammi la PARTITA IVA del COMUNE DI SARONNO e della SCUOLA ALLIEVI CARABINIERI DI TORINO. Dammi anche l' indirizzo odonimo. Aggiungi descrizione delle due denomizaioni.
Thought: The current language of the user is: Italian. I need to use a tool to help me answer the question.
Action: filtra_dati_per_chiavi
Action Input: {'names': ['COMUNE DI SARONNO', 'SCUOLA ALLIEVI CARABINIERI DI TORINO'], 'chiavi': ['partita_iva', 'indirizzo_odonimo', 'descrizione']}
Observation: [{'partita_iva': '00217130129', 'indirizzo_odonimo': 'PIAZZA REPUBBLICA 7', 'descrizione': None}, {'partita_iva': '00000000000', 'indirizzo_odonimo': 'VIA CERNAIA 23', 'descrizione': None}]
> Running step 40d075d0-2e5e-4058-a3a2-0e04a22ea6dd. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: Il COMUNE DI SARONNO ha la PARTITA IVA 00217130129 e si trova in PIAZZA REPUBBLICA 7. La descrizione di qu

In [24]:
risposta

AgentChatResponse(response='Il COMUNE DI SARONNO ha la PARTITA IVA 00217130129 e si trova in PIAZZA REPUBBLICA 7. La descrizione di questa denominazione non è disponibile.\n\nLa SCUOLA ALLIEVI CARABINIERI DI TORINO ha la PARTITA IVA 00000000000 e si trova in VIA CERNAIA 23. Anche per questa denominazione la descrizione non è disponibile.', sources=[ToolOutput(blocks=[TextBlock(block_type='text', text="[{'partita_iva': '00217130129', 'indirizzo_odonimo': 'PIAZZA REPUBBLICA 7', 'descrizione': None}, {'partita_iva': '00000000000', 'indirizzo_odonimo': 'VIA CERNAIA 23', 'descrizione': None}]")], tool_name='filtra_dati_per_chiavi', raw_input={'args': (), 'kwargs': {'names': ['COMUNE DI SARONNO', 'SCUOLA ALLIEVI CARABINIERI DI TORINO'], 'chiavi': ['partita_iva', 'indirizzo_odonimo', 'descrizione']}}, raw_output=[{'partita_iva': '00217130129', 'indirizzo_odonimo': 'PIAZZA REPUBBLICA 7', 'descrizione': None}, {'partita_iva': '00000000000', 'indirizzo_odonimo': 'VIA CERNAIA 23', 'descrizione': 

In [25]:
from setting import Setting
params = Setting()
path_to_json = params.path_to_json
with open(path_to_json, "r", encoding="utf-8") as f:
        data = [json.loads(line) for line in f][:10]
df = pd.DataFrame(data)
df.head()

,codice_fiscale,partita_iva,denominazione,codice_ausa,natura_giuridica_codice,natura_giuridica_descrizione,soggetto_estero,provincia_codice,provincia_nome,citta_codice,citta_nome,indirizzo_odonimo,cap,flag_inHouse,flag_partecipata,stato,data_inizio,data_fine
0,00217130129,00217130129,COMUNE DI SARONNO,0000155476,15,ENTI PUBBLICI NON ECONOMICI,False,IT-VA,VARESE,012119,SARONNO,PIAZZA REPUBBLICA 7,21047,False,False,ATTIVO,2013-07-09,2099-12-31
1,80093190017,00000000000,SCUOLA ALLIEVI CARABINIERI DI TORINO,0000323718,10,ALTRI ENTI ED ISTITUTI CON PERSONALITA' GIURIDICA,False,IT-TO,TORINO,001272,TORINO,VIA CERNAIA 23,10121,False,False,ATTIVO,2024-10-01,2099-12-31
2,CFAVCP-0000DCC,00000000000,SUA-PE,0000545079,999,NON CLASSIFICATO,False,IT-PE,PESCARA,068028,PESCARA,PIAZZA ITALIA,65121,False,False,ATTIVO,2024-07-15,2099-12-31
3,80010270751,00000000000,61 STORMO,0000237279,15,ENTI PUBBLICI NON ECONOMICI,False,IT-LE,LECCE,075029,GALATINA,VIALE DELL'AEREONAUTICA 1,73013,False,False,ATTIVO,2013-07-09,2099-12-31
4,03334231200,03334231200,COMUNE VALSAMOGGIA,0000368741,55,AMMINISTRAZIONI PUBBLICHE,False,IT-BO,BOLOGNA,037061,VALSAMOGGIA,PIAZZA GARIBALDI 1,40050,False,False,ATTIVO,2024-09-30,2099-12-31
